# Species - Plant Resources of South East Asia

In [17]:
from bs4 import BeautifulSoup
import requests
import json,codecs
import pandas as pd
from slugify import slugify

# Step 1: Open and read the HTML file
html_file_path = "data/SEA-species.html"  # Replace with your HTML file path
with open(html_file_path, "r", encoding="utf-8") as file:
    html_content = file.read()

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find the <main> tag
# main_tag = soup.find("main")

# Find the <div> tag with class "container"
container_divs = soup.find_all("article", class_="species-card")

# Find all <a> tags
a_tags = []
data = []
for container_div in container_divs:
    # a_tags += container_div.find_all("a")
    try:
        
        labels = [x.text.lower().strip() for x in container_div.find_all("div", class_="pn-labeled-data-label")]
        contents = [x.text.lower().strip() for x in container_div.find_all("div", class_="pn-labeled-data-content")]
        merged_dict = dict(zip(labels, contents))
        original = {
            "common name":"",
            "genus":"",
            "family":"",
        }
        merged_dict = original | merged_dict  # Merge dicts (dict2 values overwrite dict1)
        card = {
            "species-name" : container_div.find("h1").text.strip(),
            "slug" : slugify(container_div.find("h1").text.strip()),
            "common-name" : merged_dict["common name"],
            "common-name-th" : "",
            "genus" : merged_dict["genus"],
            "family" : merged_dict["family"],
            "images" : [ x.get("src") for x in container_div.find_all("img")],
            "num-images" : container_div.find_all("span")[-2].text.replace(",", "").strip(),
            "num-observations" : container_div.find_all("span")[-1].text.replace(",", "").split(" ")[0].strip(),
        }
        data.append(card);
    except Exception as e:
        print(f"An error occurred: {e}")
        print(container_div.find("h1").text.strip())
    
    # break;

#data
df = pd.DataFrame(data)
# df["common-name-th"] = df["common-name"]


filename = "data/SEA-species.xlsx"
df.to_excel(filename, index=False)  # บันทึกเป็น Excel


In [18]:
import pandas as pd

# Load JSON file into DataFrame
filename = "data/SEA-species.csv"
df = pd.read_csv(filename)

# แปลง DataFrame เป็น JSON
json_file = "data/SEA-species.json"
df.to_json(json_file, orient="records", lines=False, force_ascii=False, indent=4)

In [ ]:
from googletrans import Translator
import pandas as pd
import numpy as np
import time

# Load JSON file into DataFrame
filename = "data/SEA-species.json"
df = pd.read_json(filename)

#Translator
translator = Translator()

#Translate
texts = list(df["common-name"])

# Create an example array of 1000 strings (or any multiple of 100)
texts = np.array(texts)
texts = np.pad(texts,(0,62),constant_values=-1)

# Reshape array to have 100 elements per row
reshaped_array = texts.reshape(-1, 100)


answers = []
for index, row in enumerate(reshaped_array):
    translations  = await translator.translate(row.tolist(), dest = 'th')
    answers = answers + [x.text for x in translations]
    print(index)
    # Sleep for 5 seconds
    time.sleep(5)
    # break

texts = np.array(answers)
filtered_list = texts[texts != "-1"].tolist()

df["common-name-th"] =  filtered_list

# แปลง DataFrame เป็น JSON
json_file = "data/SEA-species-th.json"
df.to_json(json_file, orient="records", lines=False, force_ascii=False, indent=4)

filename = "data/SEA-species-th.xlsx"
df.to_excel(filename, index=False)  # บันทึกเป็น Excel